In [3]:
import jpeg_toolbox

In [34]:
img = jpeg_toolbox.load('./src/image/input.jpeg')
img2 = jpeg_toolbox.load('./src/image/output.jpeg')

In [35]:
img['coef_arrays'][0][0][8]

np.float64(99.0)

In [36]:
img2['coef_arrays'][0][0][8]

np.float64(99.0)

In [93]:
img['coef_arrays'][1]

array([[-17.,   0.,   0., ...,   0.,   0.,   0.],
       [  2.,   0.,   0., ...,   0.,   0.,   0.],
       [  0.,   0.,   0., ...,   0.,   0.,   0.],
       ...,
       [  0.,   0.,   0., ...,   0.,   0.,   0.],
       [  0.,   0.,   0., ...,   0.,   0.,   0.],
       [  0.,   0.,   0., ...,   0.,   0.,   0.]], shape=(1000, 1504))

In [94]:
img2['coef_arrays'][1]

array([[-17.,   0.,   0., ...,   0.,   0.,   0.],
       [  2.,   0.,   0., ...,   0.,   0.,   0.],
       [  0.,   0.,   0., ...,   0.,   0.,   0.],
       ...,
       [  0.,   0.,   0., ...,   0.,   0.,   0.],
       [  0.,   0.,   0., ...,   0.,   0.,   0.],
       [  0.,   0.,   0., ...,   0.,   0.,   0.]], shape=(1000, 1504))

In [95]:
img['coef_arrays'][2]

array([[13.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ...,
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]], shape=(1000, 1504))

In [96]:
img2['coef_arrays'][2]

array([[13.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ...,
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]], shape=(1000, 1504))

In [4]:
img["quant_tables"][0]


array([[ 6,  4,  4,  6,  9, 14, 18, 22],
       [ 4,  4,  5,  7,  9, 21, 22, 20],
       [ 5,  5,  6,  9, 14, 21, 25, 20],
       [ 5,  6,  8, 10, 18, 31, 29, 22],
       [ 6,  8, 13, 20, 24, 39, 37, 28],
       [ 9, 13, 20, 23, 29, 37, 41, 33],
       [18, 23, 28, 31, 37, 44, 43, 36],
       [26, 33, 34, 35, 40, 36, 37, 36]], dtype=int32)

In [5]:
img['coef_arrays'][0][0,8]

np.float64(99.0)

In [ ]:
import os
import jpeg_toolbox
from src.program.encrip import hash_to_bits
from src.program.hash_utils import generate_hash
from src.program.db_utils import store_hash_and_message_in_db

def cifrar_custom(message, input_image_path, output_image_path):
    # [1] Generate hash
    hash_128 = generate_hash(message, bits=128)
    hash_bits = hash_to_bits(hash_128)

    # [2] Store hash and message
    store_hash_and_message_in_db(hash_128, message)

    # [3] Load image
    img = jpeg_toolbox.load(input_image_path)
    quant_table = img['quant_tables'][0]
    quant_00 = quant_table[0, 0]
    coef_array = img['coef_arrays'][0].copy()

    # [4] Embed hash
    bit_idx = 0
    for i in range(0, coef_array.shape[0], 8):
        for j in range(0, coef_array.shape[1], 8):
            if bit_idx >= len(hash_bits):
                break
            block = coef_array[i:i+8, j:j+8]
            if block.shape != (8, 8):
                continue
            original_coef = int(block[0, 0])
            quantized = original_coef * quant_00
            is_negative = quantized < 0
            bin_value = list(bin(abs(quantized))[2:].zfill(12))
            for k in range(4):
                if bit_idx + k < len(hash_bits):
                    bin_value[-4 + k] = str(hash_bits[bit_idx + k])
            bit_idx += 4
            new_value = int(''.join(bin_value), 2)
            if is_negative:
                new_value = -new_value
            new_coef = new_value // quant_00
            block[0, 0] = new_coef
            coef_array[i:i+8, j:j+8] = block
        if bit_idx >= len(hash_bits):
            break

    # [5] Save modified image
    img['coef_arrays'][0] = coef_array
    jpeg_toolbox.save(img, output_image_path)

# Main loop to generate 1000 entries
input_image = './src/image/input.jpeg'
for i in range(1000):
    message = f"Test message number {i}"
    os.environ["MESSAGE"] = message
    output_image = f"./src/image/output_{i}.jpeg"
    print(f"[{i}] Processing and saving to {output_image}")
    cifrar_custom(message, input_image, output_image)
